Using meta-llama/Meta-Llama-3-8B since this is not fine tuned for question answering and is a small enough model for our use

Cloning into 'Meta-Llama-3-8B'...
remote: Access to model meta-llama/Meta-Llama-3-8B is restricted and you are not in the authorized list. Visit https://huggingface.co/meta-llama/Meta-Llama-3-8B to ask for access.
fatal: unable to access 'https://huggingface.co/meta-llama/Meta-Llama-3-8B/': The requested URL returned error: 403


In [ ]:
!pip install bitsandbytes datasets accelerate loralib -q
!pip install git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 4.5 MB/s eta 0:00:00
ERROR: Operation cancelled by user


In [ ]:
from huggingface_hub import notebook_login
notebook_login()
# token = ''

In [4]:
!nvidia-smi

Thu Jul  4 17:40:51 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
import warnings
import pandas as pd
# warnings.filterwarnings("ignore")
import os
from datasets import load_dataset
import time
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

torch.random.manual_seed(0)

In [3]:
ds = load_dataset("nvidia/HelpSteer2")
train = ds['train'] # len(train) = 20324 (95%)
val = ds['validation']     # len(val) = 1038 (5%)

train_pd = pd.DataFrame(train)
val_pd = pd.DataFrame(val)
train_pd.head()
val_pd.head()

,prompt,response,helpfulness,correctness,coherence,complexity,verbosity
0,explain master slave replication nsql,"In the context of NoSQL databases, master-slav...",4,4,4,3,2
1,explain master slave replication nsql,"In SQL, master-slave replication is a techniqu...",2,3,3,3,3
2,I am studying for a clinical informatics board...,"Sure, I can help you with that. Please provide...",4,4,4,1,1
3,I am studying for a clinical informatics board...,"Sure, I would be happy to help you with your s...",1,1,3,2,4
4,This is a set of ballots for a ranked choice e...,Here are the results of the ranked choice elec...,2,2,3,2,3


In [31]:
print(train_pd.shape), print(val_pd.shape)

(20324, 7)
(1038, 7)


(None, None)

### 1.	Perform necessary data preprocessing techniques :

LLMS can directly imput text data, prompt col already works, no need for more processing

### 2.	Explain and implement how LLM (2B to 8B parameters) over simple LM can help improve the system accuracy

We use gpt-2 (124 M) as lm, meta-llama/Meta-Llama-3-8B as llm, use rouge score to compare outputs of both models. Ground truth is response column

In [23]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AutoTokenizer, AutoModelForCausalLM

# Initialize GPT-2 tokenizer and model
lm_tokenizer = GPT2Tokenizer.from_pretrained("openai-community/gpt2")
lm_model = GPT2LMHeadModel.from_pretrained("openai-community/gpt2",
                                        pad_token_id=tokenizer.eos_token_id)

# Function to generate response given a prompt
def generate_response(text_):
    input_ids = lm_tokenizer.encode(text_, return_tensors='pt')
    output = lm_model.generate(input_ids, max_new_tokens=80, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
    response = lm_tokenizer.decode(output[0], skip_special_tokens=True)
    return response

# Example prompt
ip = "Once upon a time"

# Generate response
response = generate_response(ip)

print(f"Prompt: {ip}")
print(f"Response: {response}")


Prompt: Once upon a time
Response: Once upon a time, the world was a place of great beauty and great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and


In [36]:
sample_data = val_pd[:200]
sample_data.head(), sample_data.shape

(                                              prompt  \
 0              explain master slave replication nsql   
 1              explain master slave replication nsql   
 2  I am studying for a clinical informatics board...   
 3  I am studying for a clinical informatics board...   
 4  This is a set of ballots for a ranked choice e...   
 
                                             response  helpfulness  \
 0  In the context of NoSQL databases, master-slav...            4   
 1  In SQL, master-slave replication is a techniqu...            2   
 2  Sure, I can help you with that. Please provide...            4   
 3  Sure, I would be happy to help you with your s...            1   
 4  Here are the results of the ranked choice elec...            2   
 
    correctness  coherence  complexity  verbosity  
 0            4          4           3          2  
 1            3          3           3          3  
 2            4          4           1          1  
 3            1          3

In [37]:
init = time.time()
sample_data['lm_response'] = sample_data['prompt'].apply(generate_response)
sample_data.head()
sample_data.to_parquet('sample_pd_gpt2.parquet')
(time.time() - init)/60

970.358199596405

In [29]:
llm_model_name = "meta-llama/Meta-Llama-3-8B"
llm_tokenizer = AutoTokenizer.from_pretrained(llm_model_name, add_bos_token=False, add_eos_token=True)
llm_model = AutoModelForCausalLM.from_pretrained(llm_model_name, add_bos_token=False, add_eos_token=True)

def generate_llm_responses(text_):
    inputs = llm_tokenizer.encode_plus(text_, return_tensors='pt', padding=True, truncation=True)
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    outputs = llm_model.generate(input_ids, attention_mask=attention_mask, max_length=140,
                                 num_return_sequences=1, pad_token_id=llm_tokenizer.eos_token_id)
    response = llm_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Meta-Llama-3-8B.
403 Client Error. (Request ID: Root=1-6686f42d-6ebb12ed2d15fbac5173ed90;b1a5d7da-6aee-41f3-98b0-e7f5a380a983)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3-8B/resolve/main/config.json.
Access to model meta-llama/Meta-Llama-3-8B is restricted and you are not in the authorized list. Visit https://huggingface.co/meta-llama/Meta-Llama-3-8B to ask for access.

In [26]:
# Generate response
llm_response = generate_llm_responses(ip)
print(f"Prompt: {ip}")
print(f"Response: {llm_response}")

NameError: name 'generate_llm_responses' is not defined

In [ ]:
# Function to evaluate the predictions (This should be customized based on your specific evaluation criteria)
def evaluate_predictions(predictions, ground_truths):
    # Example placeholder for evaluation logic
    # You should implement your own evaluation metric, e.g., BLEU, ROUGE, accuracy, etc.
    pass

# Example ground truth data (for evaluation)
ground_truths = [
    "The car is approaching an intersection safely.",
    "The pedestrian is crossing the road cautiously.",
    "The vehicle needs to turn left at the next junction smoothly."
]

# Evaluate predictions (Placeholder)
evaluate_predictions(predictions, ground_truths)

In [1]:
import gensim.downloader as api
import numpy as np
import pandas as pd

#Load Pretrained Glove Embedding
glove_wiki = api.load("glove-wiki-gigaword-50")


def get_GloVe(text, size, vectors, aggregation='mean'):
    vec = np.zeros(size).reshape((1, size)) # create for size of glove embedding and assign all values 0
    count = 0
    for word in text.split():
        print('\n word:   ', word)
        try:
            print('golve vect',vectors[word])
            vec += vectors[word].reshape((1, size)) # update vector with new word
            count += 1 # counts every word in sentence
        except KeyError:
            continue
    if aggregation == 'mean':
        if count != 0:
            vec /= count  #get average of vector to create embedding for sentence
        return vec
    elif aggregation == 'sum':
        return vec



[==================================================] 100.0% 66.0/66.0MB downloaded


In [1]:
df =  pd.read_csv('train_data.csv')
x_train = df['clean_text'] # preprocessed text
train_vecs_glove_mean_wiki = scale(np.concatenate([get_GloVE(z, 50, glove_wiki,'mean') for z in x_train.astype(str)]))


NameError: name 'pd' is not defined